## Gaussian Discriminant Analysis

with varying covariance $\Sigma_1 \neq \Sigma_2 \neq \Sigma_3 ... \neq \Sigma_3$

In [3]:
import pandas as pd
import numpy as np

# To reset the printoptions
# np.set_printoptions(edgeitems=3,infstr='inf',linewidth=75, nanstr='nan', precision=8,suppress=False, threshold=1000, formatter=None)

np.set_printoptions(precision=4, suppress=True)

#np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

### For multi-class label y ={0, 1, 2, ...}

In [4]:
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
y = dataset.target
#y=(y>0).astype(int) 

target_names = list(dataset.target_names)
print(target_names)

['setosa', 'versicolor', 'virginica']


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [18]:
class GaussianDiscAnalysis: 
    def compute_phi(self, y):
        n = len(y)
        phi = dict()
        for idx in range(len(self.classes)):
            phi[idx] = (1/n) * np.sum(y==idx)
        return phi
    
    def compute_mu(self, X, y):
        mu_dict = dict()
        for idx in range(len(self.classes)):
            # Add mu for each class
            mu_dict[idx] = np.sum(X[y==idx], axis=0)/ np.sum(y==idx)
        return mu_dict

    def compute_sigma(self, X, y):
        sigma_dict = dict()
        #n = len(X)
        #y = y.reshape(-1,1)
        Xmu = X.copy()
        for idx in range(len(self.classes)):
            n = np.sum(y==idx)
            Xmu_i = Xmu[y==idx] - self.mu[idx]
            sigma = (1/n) * Xmu_i.T@Xmu_i
            sigma_dict[idx] = sigma
        return sigma_dict
    
    
    def compute_Pxyi(self, X, idx):
        """Probability of X given y"""
        m = X.shape[1]
        sigma_inv = np.linalg.inv(self.sigma[idx])
        det_sigma = np.linalg.det(self.sigma[idx])
        #mu_i = mu(X, y, idx)
        Pxi = (1/((2*np.pi)**(m/2))) \
                *(1/(det_sigma**0.5)) \
                * np.exp(- 0.5*np.sum(((X-self.mu[idx])@sigma_inv)*(X-self.mu[idx]), axis=1))
    #     Pxi = np.log(1) \
    #             - np.log((2*np.pi)**(m/2)) \
    #             - np.log(np.sqrt(det_sigma)) \
    #             - np.sum(((X-mu_i)@sigma_inv)*(X-mu_i), axis=1)
        return Pxi
    
    def fit(self, X, y):
        """Computes mean, covariance and proabilities of y (phi)"""
        self.classes = np.unique(y)
        self.mu = self.compute_mu(X, y)
        self.sigma = self.compute_sigma(X, y)
        self.phi = self.compute_phi(y)
        
    def predict_proba(self, X):
        """Computes the probability of example belonging to that class"""
        n = len(X)
        Pyi = np.zeros((n, len(self.classes)))
        
        for idx in range(len(self.classes)):
            #print(self.compute_Pxyi(X, idx))
            py_i = self.compute_Pxyi(X, idx) * self.phi[idx]
            Pyi[:, idx] = py_i
        return Pyi
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def generate_data(self, class_id, num_samples=1):
        """Generates new unseen dataset from a normal distribution
            given the mean of class and covariance
        """
        mean = self.mu[class_id]
        cov = self.sigma[class_id]
        return np.random.multivariate_normal(mean, cov, num_samples)

In [19]:
GDA = GaussianDiscAnalysis()

In [20]:
GDA.fit(X_train, y_train)

In [21]:
predictions = GDA.predict(X_test)

In [22]:
predictions

array([2, 2, 2, 1, 1, 2, 0, 0, 1, 1, 0, 2, 1, 2, 1, 0, 1, 0, 0, 2, 2, 2,
       1, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1])

In [23]:
GDA.predict_proba(X_test)[0:5]

array([[0.    , 0.    , 0.1947],
       [0.    , 0.    , 0.1722],
       [0.    , 0.    , 0.2566],
       [0.    , 0.0373, 0.    ],
       [0.    , 0.0713, 0.0014]])

In [24]:
np.sum(predictions == y_test) / len(y_test)

1.0

### Generate new data

In [33]:
new_data = GDA.generate_data(class_id=1, num_samples=20)

In [34]:
new_data

array([[5.4804, 2.3352, 3.743 , 1.1628],
       [5.9854, 2.5037, 3.9809, 0.9537],
       [6.0278, 2.6524, 4.093 , 1.1597],
       [5.8079, 3.3356, 4.0447, 1.4817],
       [6.3876, 2.8588, 4.4784, 1.5099],
       [6.3737, 2.5166, 5.1192, 1.3362],
       [5.5887, 2.4597, 4.0897, 1.516 ],
       [7.1906, 2.8857, 5.7244, 1.5987],
       [5.4716, 2.0777, 3.3261, 1.0162],
       [5.9089, 2.5441, 4.4017, 1.3938],
       [5.1635, 3.0062, 4.1002, 1.3691],
       [5.7153, 3.189 , 4.4773, 1.4478],
       [6.0914, 2.7472, 4.1508, 1.2791],
       [5.0018, 1.7628, 3.4163, 0.7081],
       [6.3056, 2.6498, 4.2685, 1.346 ],
       [5.9816, 3.0611, 4.5905, 1.4894],
       [5.8354, 2.5924, 4.1674, 1.3646],
       [5.69  , 2.4421, 3.3969, 1.0013],
       [4.8994, 2.3724, 3.0546, 0.9266],
       [5.3937, 2.7395, 3.7926, 1.0605]])

In [35]:
GDA.predict(new_data)

array([1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])